In [1]:
import time
import re
import codecs
import urllib2 as ul
import xml.etree.ElementTree as ET

def ReadSent(pageno,src='../han/'):
    dat=codecs.open(src+str(pageno)+'.d',encoding='utf-8')
    dat = dat.read()
    dat = Clean(dat)
    dat = dat.replace('?','.')
    dat = dat.replace('!','.')
    return [' '.join(i.split()) for i in dat.split('.')]
    
def Clean(sentence,comma=0):
    if comma==0:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>','\r',u'▶',u'◀']:
            sentence=sentence.replace(i,' ')
    else:
        for i in ['\n','_','-','(',')','"','\'',u'▲','...','[',']',u'■','<','>',',','\r']:
            sentence=sentence.replace(i,' ')
    return sentence.strip()

def LoadDB():
    sent1=[]
    f = open('../lists/han.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]))
        for j in tmp:
            sent1.append(j)

    sent2=[]
    f = open('../lists/dat.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='../dat/')
        for j in tmp:
            sent2.append(j)
        
    sent3=[]
    f = open('../lists/oro.list')
    for line in f:
        tmp=ReadSent(int(line.split('.')[0]),src='../oro/')
        for j in tmp:
            sent3.append(j)

    return sent1+sent2+sent3

def ShortSent(sent):
    return sorted(sent, key = lambda x: len(x))

def SearchDB(sent,token):
    matches = [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def SearchDBList(sent,tokens):
    matches=[]
    for token in tokens:
        matches += [i for i in sent if i.find(token)>0]
    return sorted(matches, key=lambda x: len(x))

def Simplify(sent):
    return list(set(sent))

#a la https://github.com/mouuff/Google-Translate-API/blob/master/python/GoogleTranslate.py
def translate(to_translate, to_langage="auto", langage="auto"):
    to_translate=to_translate.encode('utf-8')
    agents = {'User-Agent':"Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1;SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727; .NET CLR 3.0.04506.30)"}
    before_trans = 'class="t0">'
    link = "http://translate.google.com/m?hl=%s&sl=%s&q=%s"% (to_langage, langage, to_translate.replace(" ", "+"))
    request = ul.Request(link, headers=agents)
    page = ul.urlopen(request).read()
    result = page[page.find(before_trans)+len(before_trans):]
    result = result.split("<")[0]
    return result

def LoadWordBank(wbank,path='./xml/'):
    tree = ET.parse(path+wbank)
    root = tree.getroot()
    mywords={}
    for j,k in zip(root.iter('Word'),root.iter('English')):
        mywords[j.text]=k.text
    return mywords

def WCount(sent,write=0,n=2000,name='freq.wlist'):
    mywords={}
    for i in sent:
        i=i.replace(',','')
        for j in i.split():
            try:
                mywords[j]+=1
            except KeyError:
                mywords[j]=1
    freq = sorted(mywords.keys(), key=lambda x: mywords[x])
    freq.reverse()
    if write==1:
        dat=codecs.open('./output/'+name,'w',encoding='utf-8')
        for i in freq[:n]:
            try:
                dat.write(i)
                dat.write(', '+translate(i))
                dat.write('\n')
            except UnicodeDecodeError:
                continue
        
    return mywords, freq
        
def LoadFreqList(name,src='./output/'):
    f = open(src+name)
    keys=[]
    vals=[]
    for line in f:
        tmp = line.decode('utf-8').split(',')
        keys.append(tmp[0])
        vals.append(int(tmp[1]))
    return keys,vals

def MergeFreqList(name1,name2):
    dic1={}
    keys,vals=LoadFreqList(name1)
    keys1,vals1=LoadFreqList(name2)
    for i,j in zip(keys,vals):
        dic1[i]=j
    for i,j in zip(keys1,vals1):
        try:
            dic1[i]+=j
        except KeyError:
            dic1[i]=j
    keyt=sorted(dic1.keys(), key=lambda x: dic1[x])
    keyt.reverse()
    valt=[dic1[i] for i in keyt]
    return keyt,valt

In [2]:
sent = LoadDB()

##KoreanClass101 Lower Beginner #13 'Finding Something to Eat in Korea'

In [15]:
# #/배
# bae stomach, belly noun
# 아주
# aju very adverb
# 불고기 bulgogi Bulgogi, grilled noun marinated meat
# 죽다
# jukda to die verb
# 식사하다
# siksahada to eat a meal verb
# 고프다
# gopeuda to be hungry verb
# 먹다
# meoktta to eat verb
# 지금
# jigeum now adverb
# 같이
# gachi together adverb
# 정말
# jeongmal really, for real adverb

tags = [u'겠다 ',u'겠어요',u'을래',u' 배 ',u' 아주 ',u' 식사하다 ',u' 고프다 ',u' 먹다 ',u' 지금 ',u' 같이 ',u' 정말 ']
for tag in tags:
    stag = SearchDB(sent,tag)
    stag=Simplify(stag)
    stag=ShortSent(stag)
    print tag, len(stag)
    for i in range(min([10,len(stag)])):print stag[i]
    print ''
    print tag
    for i in range(min([30,len(stag)])):print stag[i],translate(stag[i])
    print ''

겠다  2085
모르겠다 웃음
노력해보겠다 웃음
잘 모르겠다 웃음
이해 못하겠다 웃음
지키겠다 쟁취하겠다
노력해봐야겠다 생각했다
글쎄 잘 모르겠다 웃음
잘 두면 이기겠다 싶었죠
열심히 두겠다 고 말했다
우승을 노리겠다 고 했다

겠다 
모르겠다 웃음 I do not laughing
노력해보겠다 웃음 I&#39;ll try smile
잘 모르겠다 웃음 Well I do not laughing
이해 못하겠다 웃음 I will not understand smile
지키겠다 쟁취하겠다 I&#39;ll keep win
노력해봐야겠다 생각했다 It brings trying thought
글쎄 잘 모르겠다 웃음 Well I do not know smile
잘 두면 이기겠다 싶었죠 I wanted to be well leave
열심히 두겠다 고 말했다 Said hard dugetda
우승을 노리겠다 고 했다 Nori won I had high
시간이 지나봐야겠다 웃음 It brings laughter over time
여기 참 좋겠다 싶더군요 Here I deogunyo want true hope
은행에 넣어 두겠다 웃음 Put smiles on the banks dugetda
미치겠다 며 엄살 로 응수 Reply, said baby, it&#39;s crazy
최선을 다하겠다 고 말했다 He said, will do my best
하지만 아직 모르겠다 웃음 But still I do not laughing
제발 이겼으면 좋겠다 웃음 Please wish won laugh
대로 준비하겠다 고 말했다 I will prepare as he said
열심히 두어야겠다 고 말했다 I said I should work hard
알 수 없다, 모르겠다 였다 I can not know and was not sure
결과에 승복하겠다 고 말했다 Will result in robes he said,
정도로 해주셨으면 좋겠다 웃음 If I&#39;d laugh so haejusyeot
나도 